# Example: Properties

#### Imports

In [30]:
import numpy as np
import torch
import time
from pathlib import Path

import dxtb

#### Functions

#### dxtb calculation

In [31]:
f = Path(globals()['_dh'][0]) / "molecules" / "taxol.coord"
atoms, xyz = dxtb.io.read_coord(f)

In [32]:
device = torch.device("cpu")
dd: dxtb.typing.DD = {"device": device, "dtype": torch.double}

numbers = torch.tensor(atoms, device=device)
positions = torch.tensor(xyz, **dd)
charge = torch.tensor(0.0, **dd)
par = dxtb.GFN1_XTB

In [33]:
import logging

logging.getLogger().setLevel(logging.ERROR)

### Autograd utils

In [34]:
repeats = 10

def nth_derivative(f, wrt, n=1):
    if n == 0:
        return
    
    create_graph = False if n == 1 else True
    
    for _ in range(n):
        grads, = torch.autograd.grad(f, wrt, create_graph=create_graph)
        f = grads.sum()

    return grads # type: ignore

def print_average(times):
    print()
    if repeats > 1:
        avg_time = sum(times) / len(times)
        print(f"Average ({repeats}): {avg_time:2.4f} sec")

def time_operation(operation, n):
    times = []
    for _ in range(repeats):
        ts = time.perf_counter()
        s = operation().sum()  # Execute the operation passed as an argument
        ds = nth_derivative(s, positions, n=n)
        te = time.perf_counter()
        times.append(te - ts)
        print(f"{te - ts:2.4f}", end=" ")
    
    print_average(times)
    
def eval(c, num, pos, chrg):
    result = c.singlepoint(num, pos, chrg)
    return result.total.sum()

# Libcint

In [35]:
opts = {"int_driver": "libcint", "verbosity": 0}
calc_lc = dxtb.Calculator(numbers, par, opts=opts)

### Energy

In [36]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_lc, numbers, positions, charge), 0)

1.4031 

KeyboardInterrupt: 

### 1st Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_lc, numbers, positions, charge), 1)

1.5768 1.5383 1.5267 1.5973 1.5994 1.5767 1.5134 1.5493 1.5329 1.5184 
Average (10): 1.5529 sec


### 2nd Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_lc, numbers, positions, charge), 2)

2.0975 2.1389 2.1188 2.2312 2.1927 2.1208 2.1229 2.1432 2.1797 2.1356 
Average (10): 2.1481 sec


### 3rd Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_lc, numbers, positions, charge), 3)

KeyboardInterrupt: 

# PyTorch

In [ ]:
opts = {"int_driver": "pytorch", "verbosity": 0}
calc_py = dxtb.Calculator(numbers, par, opts=opts)

### Energy

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_py, numbers, positions, charge), 0)

2.1724 2.0728 2.0650 2.0600 2.1237 2.0597 2.0706 2.0717 2.0759 2.1443 
Average (10): 2.0916 sec


### 1st Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_py, numbers, positions, charge), 1)

2.6598 2.6136 2.5689 2.5530 2.5907 2.6402 2.6323 2.6137 2.6947 2.5692 
Average (10): 2.6136 sec


### 2nd Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_py, numbers, positions, charge), 2)

3.3231 3.4583 3.4005 3.3169 3.2235 3.2767 3.3835 3.2916 3.3325 3.3025 
Average (10): 3.3309 sec


### 3rd Derivative

In [ ]:
positions = positions.detach().clone().requires_grad_(True)
time_operation(lambda: eval(calc_py, numbers, positions, charge), 3)

KeyboardInterrupt: 